In [ ]:
import numpy as np
from collections import defaultdict

# ============================= Robust TRIM Estimator ==========================
def trimmed_mean_estimator(data, epsilon, delta):
    """
    Two-split clipped mean:
      1) Split into D1, D2
      2) Sort D1 and pick clipping bounds via zeta(eps, delta, M)
      3) Clip D2 to [alpha, beta] and average
    """
    M = len(data)
    if M < 2:
        return np.mean(data) if M > 0 else 0.0
    if M % 2 != 0:
        data = data[:-1]
        M -= 1

    np.random.shuffle(data)
    D1, D2 = np.split(data, 2)
    D1_sorted = np.sort(D1)

    # Clipping mass parameter
    zeta = 8 * epsilon + 24 * np.log(4 / delta) / M

    li = int(np.floor(zeta * len(D1_sorted)))
    ui = int(np.floor((1 - zeta) * len(D1_sorted)) - 1)
    li = max(0, min(li, len(D1_sorted) - 1))
    ui = max(0, min(ui, len(D1_sorted) - 1))

    alpha_clip = D1_sorted[li]
    beta_clip  = D1_sorted[ui]
    trimmed    = np.clip(D2, alpha_clip, beta_clip)

    return (2 / M) * np.sum(trimmed)

# ============================= Reward Generator ==============================
def generate_reward(true_r, corruption_prob=0.0,noise_var=1.0, attack_value=-1e4):
    """
    Huber-style corruption: with prob ε, return a fixed adversarial value.
    Otherwise return true_r + N(0, noise_var).
    """
    if np.random.rand() < corruption_prob:
        return attack_value
    return true_r + np.random.normal(0.0, np.sqrt(noise_var))

# ============================= Behavior Policy / Kernel ======================
def behavior_probs_uniform(S, A):
    """Uniform behavior over actions: pi(a|s) = 1/A."""
    return np.full((S, A), 1.0 / A)

def induced_state_kernel(P, pi):
    """
    Build P_pi[s, s'] = sum_a pi[s,a] * P[s, s', a]
    P: (S, S, A); pi: (S, A)
    """
    S, _, A = P.shape
    P_pi = np.zeros((S, S))
    for s in range(S):
        for a in range(A):
            P_pi[s, :] += pi[s, a] * P[s, :, a]
    # numeric hygiene
    P_pi = np.clip(P_pi, 0.0, 1.0)
    row_sums = P_pi.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0.0] = 1.0
    P_pi = P_pi / row_sums
    return P_pi

# ============================= Stationary Distribution =======================
def stationary_distribution_power(P_pi, max_iter=10000, tol=1e-12):
    """
    Power method for left eigenvector of eigenvalue 1 (stationary distribution).
    """
    S = P_pi.shape[0]
    x = np.full(S, 1.0 / S)
    PT = P_pi.T
    for _ in range(max_iter):
        x_new = PT @ x
        s = x_new.sum()
        if s <= 0:
            x_new = np.full(S, 1.0 / S)
        else:
            x_new = x_new / s
        if np.linalg.norm(x_new - x, 1) < tol:
            return x_new
        x = x_new
    return x

# ============================= Mixing Time Estimation ========================
def spectral_mixing_time(P_pi, pi_stat, eps_mix=1e-2):
    """
    Spectral bound using SLEM (second largest eigenvalue modulus):
      t_mix(eps) ~ ceil( log(1/(eps*pi_min)) / (1 - lambda_star) )
    Returns (t_mix, lambda_star, gap).
    """
    vals = np.linalg.eigvals(P_pi)
    mags = np.abs(vals)
    order = np.argsort(-mags)         # descending by magnitude
    lambda_star = mags[order[1]] if len(vals) > 1 else 0.0  # SLEM
    gap = max(1e-8, 1.0 - lambda_star)
    pi_min = max(1e-12, float(np.min(pi_stat)))
    t_mix = int(np.ceil(np.log(1.0 / (eps_mix * pi_min)) / gap))
    return max(1, t_mix), float(lambda_star), float(gap)

def empirical_mixing_time(P_pi, pi_stat, eps_mix=1e-2, T_max=2_000_000,
                          burn=10_000, check_every=1000, seed=123):
    """
    Empirical fallback: simulate and find first t with TV(empirical, pi_stat) < eps_mix.
    """
    rng = np.random.default_rng(seed)
    S = P_pi.shape[0]
    s = rng.integers(S)
    counts = np.zeros(S, dtype=np.int64)

    for t in range(1, T_max + 1):
        s = rng.choice(S, p=P_pi[s])
        counts[s] += 1

        if t >= burn and t % check_every == 0:
            emp = counts / counts.sum()
            tv = 0.5 * np.abs(emp - pi_stat).sum()
            if tv < eps_mix:
                return t
    return T_max

def estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=1e-2):
    """
    Estimate bar_tau = t_mix(eps_mix) for the behavior-policy–induced chain.
    Try spectral; fall back to empirical if needed.
    """
    if behavior == 'uniform':
        pi = behavior_probs_uniform(S, A)
    else:
        raise NotImplementedError("Only 'uniform' behavior is implemented here.")

    P_pi = induced_state_kernel(P, pi)
    pi_stat = stationary_distribution_power(P_pi)

    try:
        t_spec, slem, gap = spectral_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)
        # If the gap is tiny, confirm with an empirical probe (conservative)
        if gap < 1e-6 or 1.0 - gap > 0.999999:
            t_emp = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix,
                                          T_max=min(2_000_000, 200 * t_spec))
            bar_tau = t_emp if t_emp > 0 else t_spec
        else:
            bar_tau = t_spec
    except Exception:
        bar_tau = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)

    return int(max(1, bar_tau)), pi_stat

# ============================= Tau / Threshold ===============================
def compute_tau(T, delta, bar_tau):
    """
    tau = floor( ceil(log(2T/delta)/log 2) * bar_tau )
    """
    return int(np.floor(np.ceil(np.log(2 * T / delta) / np.log(2)) * bar_tau))

def G_t(t, eps, p_hat, R, noise_var, C, T, ra_mod_delta):
    """
    \tilde{G}_t with early-iteration guard and scaling in sqrt(log/dt) + sqrt(eps).
    """
    T_lim = (104 / 3 * p_hat) * np.log((800 * T) / ra_mod_delta) + 1
    if t <= T_lim:
        return 0.0
    else:
      term = np.sqrt((4.0 * np.log(4.0 / ra_mod_delta)) / (3.0 * p_hat * max(t, 1.0))) + np.sqrt(eps)
      return C * (t**1) * term + (t**1)

# ============================= Load Data =====================================
data   = np.load('Qlearn_data_new.npz')
P      = data['P']         # (S, S, A)
R      = data['R']         # (S, A)
Qopt   = data['Qopt']      # optimal Q
gamma  = data['gamma'].item()
S      = data['S'].item()
A      = data['A'].item()
p      = data['p']         # visitation lower-bounds proxy (vector)
p_hat  = np.min(p)

# ============================= Hyperparameters ===============================
T            = 25000000                 # single long trajectory length
delta        = 0.1
C            = 1.1
alpha        = 0.1
epsilons     = [0.001]
noise_var    = 5.0                     # Change accordingly
attack_value = -1e4
ra_mod_delta = (delta**2) / (320000 * (T**(5)))         # δ/(4T) for TRIM
eps_mix      = 1e-2                    # TV threshold for mixing-time estimate
window       = int(0.01 * T)           # steady-state averaging window

# ============================= Estimate bar_tau and tau ======================
bar_tau, pi_stat = estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=eps_mix)
tau = compute_tau(T, delta, bar_tau)
print(f"[mixing-time] eps_mix={eps_mix:.1e}, bar_tau≈{bar_tau}, tau={tau}, T={T}")

# ============================= Behavior Policy ===============================
def behavior_action(s):
    """Uniform behavior policy over actions."""
    return np.random.randint(A)

# ============================= Single-Trajectory Run =========================
# One error curve per epsilon (no averaging over runs)
error_matrix = np.zeros((len(epsilons), T))

for idx, eps in enumerate(epsilons):
    Q   = np.zeros((S, A))
    Err = np.zeros(T)

    # Initialize chain
    s_t = 0
    D   = defaultdict(list)  # buffers per (s,a), appended only at subsampled indices

    for t in range(T):
        # (i) action from behavior policy
        a_t = behavior_action(s_t)

        # (ii) transition: Markovian via P[s_t, :, a_t]
        s_tp1 = np.random.choice(S, p=P[s_t, :, a_t])

        # (iii) reward under Huber contamination
        y_t = generate_reward(R[s_t, a_t],
                              corruption_prob=eps,
                              noise_var=noise_var,
                              attack_value=attack_value)

        if t % tau == 0:
            # --- subsampled robust update ---
            # (iv) append reward to dataset for current (s,a)
            D[(s_t, a_t)].append(y_t)

            # (v) robust TRIM estimate with ε and δ/(4T)
            r_bar = trimmed_mean_estimator(
                np.array(D[(s_t, a_t)]), epsilon=eps, delta=ra_mod_delta
            )

            # (vi) thresholding via \tilde{G}_t
            Gval = G_t(t=t, eps=eps, p_hat=p_hat, R=R, noise_var=noise_var,
                       C=C, T=T, ra_mod_delta=ra_mod_delta)
            r_tilde = 0.0 if abs(r_bar) > Gval else r_bar

            # (vii) robust Q-update
            Q[s_t, a_t] = (1 - alpha) * Q[s_t, a_t] + alpha * (
                r_tilde + gamma * np.max(Q[s_tp1])
            )
        # else: no update; carry Q forward

        # (viii) log sup-norm error vs. Q*
        Err[t] = np.max(np.abs(Q - Qopt))

        # advance chain
        s_t = s_tp1

    error_matrix[idx, :] = Err

    # steady-state summary
    final_err = Err[-1]
    avg_tail  = float(np.mean(Err[-window:]))
    print(f"ε = {eps:<6} | E_T = {final_err:.6f} | mean(last {window:,}) = {avg_tail:.6f}")

# ============================= Save Result ===================================
out_name = f'Robust_RAQ_error_markov_singletraj_var_{int(noise_var)}_tau_adaptive_eps_0.001.npy'
np.save(out_name, error_matrix)
print(f"\nSaved error traces to '{out_name}' (shape = [len(epsilons), T]).")


In [ ]:
import numpy as np
from collections import defaultdict

# ============================= Robust TRIM Estimator ==========================
def trimmed_mean_estimator(data, epsilon, delta):
    """
    Two-split clipped mean:
      1) Split into D1, D2
      2) Sort D1 and pick clipping bounds via zeta(eps, delta, M)
      3) Clip D2 to [alpha, beta] and average
    """
    M = len(data)
    if M < 2:
        return np.mean(data) if M > 0 else 0.0
    if M % 2 != 0:
        data = data[:-1]
        M -= 1

    np.random.shuffle(data)
    D1, D2 = np.split(data, 2)
    D1_sorted = np.sort(D1)

    # Clipping mass parameter
    zeta = 8 * epsilon + 24 * np.log(4 / delta) / M

    li = int(np.floor(zeta * len(D1_sorted)))
    ui = int(np.floor((1 - zeta) * len(D1_sorted)) - 1)
    li = max(0, min(li, len(D1_sorted) - 1))
    ui = max(0, min(ui, len(D1_sorted) - 1))

    alpha_clip = D1_sorted[li]
    beta_clip  = D1_sorted[ui]
    trimmed    = np.clip(D2, alpha_clip, beta_clip)

    return (2 / M) * np.sum(trimmed)

# ============================= Reward Generator ==============================
def generate_reward(true_r, corruption_prob=0.0,noise_var=1.0, attack_value=-1e4):
    """
    Huber-style corruption: with prob ε, return a fixed adversarial value.
    Otherwise return true_r + N(0, noise_var).
    """
    if np.random.rand() < corruption_prob:
        return attack_value
    return true_r + np.random.normal(0.0, np.sqrt(noise_var))

# ============================= Behavior Policy / Kernel ======================
def behavior_probs_uniform(S, A):
    """Uniform behavior over actions: pi(a|s) = 1/A."""
    return np.full((S, A), 1.0 / A)

def induced_state_kernel(P, pi):
    """
    Build P_pi[s, s'] = sum_a pi[s,a] * P[s, s', a]
    P: (S, S, A); pi: (S, A)
    """
    S, _, A = P.shape
    P_pi = np.zeros((S, S))
    for s in range(S):
        for a in range(A):
            P_pi[s, :] += pi[s, a] * P[s, :, a]
    # numeric hygiene
    P_pi = np.clip(P_pi, 0.0, 1.0)
    row_sums = P_pi.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0.0] = 1.0
    P_pi = P_pi / row_sums
    return P_pi

# ============================= Stationary Distribution =======================
def stationary_distribution_power(P_pi, max_iter=10000, tol=1e-12):
    """
    Power method for left eigenvector of eigenvalue 1 (stationary distribution).
    """
    S = P_pi.shape[0]
    x = np.full(S, 1.0 / S)
    PT = P_pi.T
    for _ in range(max_iter):
        x_new = PT @ x
        s = x_new.sum()
        if s <= 0:
            x_new = np.full(S, 1.0 / S)
        else:
            x_new = x_new / s
        if np.linalg.norm(x_new - x, 1) < tol:
            return x_new
        x = x_new
    return x

# ============================= Mixing Time Estimation ========================
def spectral_mixing_time(P_pi, pi_stat, eps_mix=1e-2):
    """
    Spectral bound using SLEM (second largest eigenvalue modulus):
      t_mix(eps) ~ ceil( log(1/(eps*pi_min)) / (1 - lambda_star) )
    Returns (t_mix, lambda_star, gap).
    """
    vals = np.linalg.eigvals(P_pi)
    mags = np.abs(vals)
    order = np.argsort(-mags)         # descending by magnitude
    lambda_star = mags[order[1]] if len(vals) > 1 else 0.0  # SLEM
    gap = max(1e-8, 1.0 - lambda_star)
    pi_min = max(1e-12, float(np.min(pi_stat)))
    t_mix = int(np.ceil(np.log(1.0 / (eps_mix * pi_min)) / gap))
    return max(1, t_mix), float(lambda_star), float(gap)

def empirical_mixing_time(P_pi, pi_stat, eps_mix=1e-2, T_max=2_000_000,
                          burn=10_000, check_every=1000, seed=123):
    """
    Empirical fallback: simulate and find first t with TV(empirical, pi_stat) < eps_mix.
    """
    rng = np.random.default_rng(seed)
    S = P_pi.shape[0]
    s = rng.integers(S)
    counts = np.zeros(S, dtype=np.int64)

    for t in range(1, T_max + 1):
        s = rng.choice(S, p=P_pi[s])
        counts[s] += 1

        if t >= burn and t % check_every == 0:
            emp = counts / counts.sum()
            tv = 0.5 * np.abs(emp - pi_stat).sum()
            if tv < eps_mix:
                return t
    return T_max

def estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=1e-2):
    """
    Estimate bar_tau = t_mix(eps_mix) for the behavior-policy–induced chain.
    Try spectral; fall back to empirical if needed.
    """
    if behavior == 'uniform':
        pi = behavior_probs_uniform(S, A)
    else:
        raise NotImplementedError("Only 'uniform' behavior is implemented here.")

    P_pi = induced_state_kernel(P, pi)
    pi_stat = stationary_distribution_power(P_pi)

    try:
        t_spec, slem, gap = spectral_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)
        # If the gap is tiny, confirm with an empirical probe (conservative)
        if gap < 1e-6 or 1.0 - gap > 0.999999:
            t_emp = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix,
                                          T_max=min(2_000_000, 200 * t_spec))
            bar_tau = t_emp if t_emp > 0 else t_spec
        else:
            bar_tau = t_spec
    except Exception:
        bar_tau = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)

    return int(max(1, bar_tau)), pi_stat

# ============================= Tau / Threshold ===============================
def compute_tau(T, delta, bar_tau):
    """
    tau = floor( ceil(log(2T/delta)/log 2) * bar_tau )
    """
    return int(np.floor(np.ceil(np.log(2 * T / delta) / np.log(2)) * bar_tau))

def G_t(t, eps, p_hat, R, noise_var, C, T, ra_mod_delta):
    """
    \tilde{G}_t with early-iteration guard and scaling in sqrt(log/dt) + sqrt(eps).
    """
    T_lim = (104 / 3 * p_hat) * np.log((800 * T) / ra_mod_delta) + 1
    if t <= T_lim:
        return 0.0
    else:
      term = np.sqrt((4.0 * np.log(4.0 / ra_mod_delta)) / (3.0 * p_hat * max(t, 1.0))) + np.sqrt(eps)
      return C * (t**1) * term + (t**1)

# ============================= Load Data =====================================
data   = np.load('Qlearn_data_new.npz')
P      = data['P']         # (S, S, A)
R      = data['R']         # (S, A)
Qopt   = data['Qopt']      # optimal Q
gamma  = data['gamma'].item()
S      = data['S'].item()
A      = data['A'].item()
p      = data['p']         # visitation lower-bounds proxy (vector)
p_hat  = np.min(p)

# ============================= Hyperparameters ===============================
T            = 25000000                 # single long trajectory length
delta        = 0.1
C            = 1.1
alpha        = 0.1
epsilons     = [0.005]
noise_var    = 5.0                     # Change accordingly
attack_value = -1e4
ra_mod_delta = (delta**2) / (320000 * (T**(5)))         # δ/(4T) for TRIM
eps_mix      = 1e-2                    # TV threshold for mixing-time estimate
window       = int(0.01 * T)           # steady-state averaging window

# ============================= Estimate bar_tau and tau ======================
bar_tau, pi_stat = estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=eps_mix)
tau = compute_tau(T, delta, bar_tau)
print(f"[mixing-time] eps_mix={eps_mix:.1e}, bar_tau≈{bar_tau}, tau={tau}, T={T}")

# ============================= Behavior Policy ===============================
def behavior_action(s):
    """Uniform behavior policy over actions."""
    return np.random.randint(A)

# ============================= Single-Trajectory Run =========================
# One error curve per epsilon (no averaging over runs)
error_matrix = np.zeros((len(epsilons), T))

for idx, eps in enumerate(epsilons):
    Q   = np.zeros((S, A))
    Err = np.zeros(T)

    # Initialize chain
    s_t = 0
    D   = defaultdict(list)  # buffers per (s,a), appended only at subsampled indices

    for t in range(T):
        # (i) action from behavior policy
        a_t = behavior_action(s_t)

        # (ii) transition: Markovian via P[s_t, :, a_t]
        s_tp1 = np.random.choice(S, p=P[s_t, :, a_t])

        # (iii) reward under Huber contamination
        y_t = generate_reward(R[s_t, a_t],
                              corruption_prob=eps,
                              noise_var=noise_var,
                              attack_value=attack_value)

        if t % tau == 0:
            # --- subsampled robust update ---
            # (iv) append reward to dataset for current (s,a)
            D[(s_t, a_t)].append(y_t)

            # (v) robust TRIM estimate with ε and δ/(4T)
            r_bar = trimmed_mean_estimator(
                np.array(D[(s_t, a_t)]), epsilon=eps, delta=ra_mod_delta
            )

            # (vi) thresholding via \tilde{G}_t
            Gval = G_t(t=t, eps=eps, p_hat=p_hat, R=R, noise_var=noise_var,
                       C=C, T=T, ra_mod_delta=ra_mod_delta)
            r_tilde = 0.0 if abs(r_bar) > Gval else r_bar

            # (vii) robust Q-update
            Q[s_t, a_t] = (1 - alpha) * Q[s_t, a_t] + alpha * (
                r_tilde + gamma * np.max(Q[s_tp1])
            )
        # else: no update; carry Q forward

        # (viii) log sup-norm error vs. Q*
        Err[t] = np.max(np.abs(Q - Qopt))

        # advance chain
        s_t = s_tp1

    error_matrix[idx, :] = Err

    # steady-state summary
    final_err = Err[-1]
    avg_tail  = float(np.mean(Err[-window:]))
    print(f"ε = {eps:<6} | E_T = {final_err:.6f} | mean(last {window:,}) = {avg_tail:.6f}")

# ============================= Save Result ===================================
out_name = f'Robust_RAQ_error_markov_singletraj_var_{int(noise_var)}_tau_adaptive_eps_0.005.npy'
np.save(out_name, error_matrix)
print(f"\nSaved error traces to '{out_name}' (shape = [len(epsilons), T]).")


In [ ]:
import numpy as np
from collections import defaultdict

# ============================= Robust TRIM Estimator ==========================
def trimmed_mean_estimator(data, epsilon, delta):
    """
    Two-split clipped mean:
      1) Split into D1, D2
      2) Sort D1 and pick clipping bounds via zeta(eps, delta, M)
      3) Clip D2 to [alpha, beta] and average
    """
    M = len(data)
    if M < 2:
        return np.mean(data) if M > 0 else 0.0
    if M % 2 != 0:
        data = data[:-1]
        M -= 1

    np.random.shuffle(data)
    D1, D2 = np.split(data, 2)
    D1_sorted = np.sort(D1)

    # Clipping mass parameter
    zeta = 8 * epsilon + 24 * np.log(4 / delta) / M

    li = int(np.floor(zeta * len(D1_sorted)))
    ui = int(np.floor((1 - zeta) * len(D1_sorted)) - 1)
    li = max(0, min(li, len(D1_sorted) - 1))
    ui = max(0, min(ui, len(D1_sorted) - 1))

    alpha_clip = D1_sorted[li]
    beta_clip  = D1_sorted[ui]
    trimmed    = np.clip(D2, alpha_clip, beta_clip)

    return (2 / M) * np.sum(trimmed)

# ============================= Reward Generator ==============================
def generate_reward(true_r, corruption_prob=0.0,noise_var=1.0, attack_value=-1e4):
    """
    Huber-style corruption: with prob ε, return a fixed adversarial value.
    Otherwise return true_r + N(0, noise_var).
    """
    if np.random.rand() < corruption_prob:
        return attack_value
    return true_r + np.random.normal(0.0, np.sqrt(noise_var))

# ============================= Behavior Policy / Kernel ======================
def behavior_probs_uniform(S, A):
    """Uniform behavior over actions: pi(a|s) = 1/A."""
    return np.full((S, A), 1.0 / A)

def induced_state_kernel(P, pi):
    """
    Build P_pi[s, s'] = sum_a pi[s,a] * P[s, s', a]
    P: (S, S, A); pi: (S, A)
    """
    S, _, A = P.shape
    P_pi = np.zeros((S, S))
    for s in range(S):
        for a in range(A):
            P_pi[s, :] += pi[s, a] * P[s, :, a]
    # numeric hygiene
    P_pi = np.clip(P_pi, 0.0, 1.0)
    row_sums = P_pi.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0.0] = 1.0
    P_pi = P_pi / row_sums
    return P_pi

# ============================= Stationary Distribution =======================
def stationary_distribution_power(P_pi, max_iter=10000, tol=1e-12):
    """
    Power method for left eigenvector of eigenvalue 1 (stationary distribution).
    """
    S = P_pi.shape[0]
    x = np.full(S, 1.0 / S)
    PT = P_pi.T
    for _ in range(max_iter):
        x_new = PT @ x
        s = x_new.sum()
        if s <= 0:
            x_new = np.full(S, 1.0 / S)
        else:
            x_new = x_new / s
        if np.linalg.norm(x_new - x, 1) < tol:
            return x_new
        x = x_new
    return x

# ============================= Mixing Time Estimation ========================
def spectral_mixing_time(P_pi, pi_stat, eps_mix=1e-2):
    """
    Spectral bound using SLEM (second largest eigenvalue modulus):
      t_mix(eps) ~ ceil( log(1/(eps*pi_min)) / (1 - lambda_star) )
    Returns (t_mix, lambda_star, gap).
    """
    vals = np.linalg.eigvals(P_pi)
    mags = np.abs(vals)
    order = np.argsort(-mags)         # descending by magnitude
    lambda_star = mags[order[1]] if len(vals) > 1 else 0.0  # SLEM
    gap = max(1e-8, 1.0 - lambda_star)
    pi_min = max(1e-12, float(np.min(pi_stat)))
    t_mix = int(np.ceil(np.log(1.0 / (eps_mix * pi_min)) / gap))
    return max(1, t_mix), float(lambda_star), float(gap)

def empirical_mixing_time(P_pi, pi_stat, eps_mix=1e-2, T_max=2_000_000,
                          burn=10_000, check_every=1000, seed=123):
    """
    Empirical fallback: simulate and find first t with TV(empirical, pi_stat) < eps_mix.
    """
    rng = np.random.default_rng(seed)
    S = P_pi.shape[0]
    s = rng.integers(S)
    counts = np.zeros(S, dtype=np.int64)

    for t in range(1, T_max + 1):
        s = rng.choice(S, p=P_pi[s])
        counts[s] += 1

        if t >= burn and t % check_every == 0:
            emp = counts / counts.sum()
            tv = 0.5 * np.abs(emp - pi_stat).sum()
            if tv < eps_mix:
                return t
    return T_max

def estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=1e-2):
    """
    Estimate bar_tau = t_mix(eps_mix) for the behavior-policy–induced chain.
    Try spectral; fall back to empirical if needed.
    """
    if behavior == 'uniform':
        pi = behavior_probs_uniform(S, A)
    else:
        raise NotImplementedError("Only 'uniform' behavior is implemented here.")

    P_pi = induced_state_kernel(P, pi)
    pi_stat = stationary_distribution_power(P_pi)

    try:
        t_spec, slem, gap = spectral_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)
        # If the gap is tiny, confirm with an empirical probe (conservative)
        if gap < 1e-6 or 1.0 - gap > 0.999999:
            t_emp = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix,
                                          T_max=min(2_000_000, 200 * t_spec))
            bar_tau = t_emp if t_emp > 0 else t_spec
        else:
            bar_tau = t_spec
    except Exception:
        bar_tau = empirical_mixing_time(P_pi, pi_stat, eps_mix=eps_mix)

    return int(max(1, bar_tau)), pi_stat

# ============================= Tau / Threshold ===============================
def compute_tau(T, delta, bar_tau):
    """
    tau = floor( ceil(log(2T/delta)/log 2) * bar_tau )
    """
    return int(np.floor(np.ceil(np.log(2 * T / delta) / np.log(2)) * bar_tau))

def G_t(t, eps, p_hat, R, noise_var, C, T, ra_mod_delta):
    """
    \tilde{G}_t with early-iteration guard and scaling in sqrt(log/dt) + sqrt(eps).
    """
    T_lim = (104 / 3 * p_hat) * np.log((800 * T) / ra_mod_delta) + 1
    if t <= T_lim:
        return 0.0
    else:
      term = np.sqrt((4.0 * np.log(4.0 / ra_mod_delta)) / (3.0 * p_hat * max(t, 1.0))) + np.sqrt(eps)
      return C * (t**1) * term + (t**1)

# ============================= Load Data =====================================
data   = np.load('Qlearn_data_new.npz')
P      = data['P']         # (S, S, A)
R      = data['R']         # (S, A)
Qopt   = data['Qopt']      # optimal Q
gamma  = data['gamma'].item()
S      = data['S'].item()
A      = data['A'].item()
p      = data['p']         # visitation lower-bounds proxy (vector)
p_hat  = np.min(p)

# ============================= Hyperparameters ===============================
T            = 25000000                 # single long trajectory length
delta        = 0.1
C            = 1.1
alpha        = 0.1
epsilons     = [0.01]
noise_var    = 5.0                     # Change accordingly
attack_value = -1e4
ra_mod_delta = (delta**2) / (320000 * (T**(5)))         # δ/(4T) for TRIM
eps_mix      = 1e-2                    # TV threshold for mixing-time estimate
window       = int(0.01 * T)           # steady-state averaging window

# ============================= Estimate bar_tau and tau ======================
bar_tau, pi_stat = estimate_mixing_time(P, S, A, behavior='uniform', eps_mix=eps_mix)
tau = compute_tau(T, delta, bar_tau)
print(f"[mixing-time] eps_mix={eps_mix:.1e}, bar_tau≈{bar_tau}, tau={tau}, T={T}")

# ============================= Behavior Policy ===============================
def behavior_action(s):
    """Uniform behavior policy over actions."""
    return np.random.randint(A)

# ============================= Single-Trajectory Run =========================
# One error curve per epsilon (no averaging over runs)
error_matrix = np.zeros((len(epsilons), T))

for idx, eps in enumerate(epsilons):
    Q   = np.zeros((S, A))
    Err = np.zeros(T)

    # Initialize chain
    s_t = 0
    D   = defaultdict(list)  # buffers per (s,a), appended only at subsampled indices

    for t in range(T):
        # (i) action from behavior policy
        a_t = behavior_action(s_t)

        # (ii) transition: Markovian via P[s_t, :, a_t]
        s_tp1 = np.random.choice(S, p=P[s_t, :, a_t])

        # (iii) reward under Huber contamination
        y_t = generate_reward(R[s_t, a_t],
                              corruption_prob=eps,
                              noise_var=noise_var,
                              attack_value=attack_value)

        if t % tau == 0:
            # --- subsampled robust update ---
            # (iv) append reward to dataset for current (s,a)
            D[(s_t, a_t)].append(y_t)

            # (v) robust TRIM estimate with ε and δ/(4T)
            r_bar = trimmed_mean_estimator(
                np.array(D[(s_t, a_t)]), epsilon=eps, delta=ra_mod_delta
            )

            # (vi) thresholding via \tilde{G}_t
            Gval = G_t(t=t, eps=eps, p_hat=p_hat, R=R, noise_var=noise_var,
                       C=C, T=T, ra_mod_delta=ra_mod_delta)
            r_tilde = 0.0 if abs(r_bar) > Gval else r_bar

            # (vii) robust Q-update
            Q[s_t, a_t] = (1 - alpha) * Q[s_t, a_t] + alpha * (
                r_tilde + gamma * np.max(Q[s_tp1])
            )
        # else: no update; carry Q forward

        # (viii) log sup-norm error vs. Q*
        Err[t] = np.max(np.abs(Q - Qopt))

        # advance chain
        s_t = s_tp1

    error_matrix[idx, :] = Err

    # steady-state summary
    final_err = Err[-1]
    avg_tail  = float(np.mean(Err[-window:]))
    print(f"ε = {eps:<6} | E_T = {final_err:.6f} | mean(last {window:,}) = {avg_tail:.6f}")

# ============================= Save Result ===================================
out_name = f'Robust_RAQ_error_markov_singletraj_var_{int(noise_var)}_tau_adaptive_eps_0.01.npy'
np.save(out_name, error_matrix)
print(f"\nSaved error traces to '{out_name}' (shape = [len(epsilons), T]).")
